## Some useful methods in spark

### Append rowid as an additional column to df

In [ ]:
def get_df_with_rowid(df):
    from pyspark.sql import Row
    from pyspark.sql.types import StructField, StructType, LongType

    row_with_index = Row(*["id"] + df.columns)

    def make_row(columns):
        def _make_row(row, uid):
            row_dict = row.asDict()
            return row_with_index(*[uid] + [row_dict.get(c) for c in columns])
        return _make_row

    f = make_row(df.columns)

    df_with_rowid = (df.rdd
      .zipWithIndex ()
      .map(lambda x: f(*x))
      .toDF(StructType([StructField("id", LongType(), False)] + df.schema.fields)))
    return df_with_rowid